In [7]:
import requests
from bs4 import BeautifulSoup
import re
import os
import pandas as pd # Dùng để đọc và hiển thị file csv/tsv
from datetime import datetime, timedelta # Dùng để xử lý dữ liệu thời gian

In [8]:
url = 'https://www.worldometers.info/coronavirus/'
r = requests.get(url)
soup = BeautifulSoup(r.text, 'html.parser')

## Get columns

In [9]:
cols = ['Country, Other', 'Total Cases', 'New Cases', 'Total Deaths', 'New Deaths', 'Total Recovered', 'New Recovered', 'Active Cases', 'Serious, Critical',
        'Tot Cases/1M pop', 'Deaths/1M pop', 'Total Tests', 'Tests/1M pop', 'Population']
len(cols)        

14

## Get rows

Ref: https://www.pluralsight.com/guides/extracting-data-html-beautifulsoup 

In [10]:
def get_table(which):
  '''which: today/yesterday/2 days ago'''
  table = soup.find('table', {"id": which})
  row_0 = table.tbody.find_all('tr', {'style': ''})[0]
  row_last = table.find('tbody', {'class': 'total_row_body body_world'})
  rows = table.tbody.find_all('tr', {'style': re.compile(r'(^$|^background)')})
  rows.insert(0, row_0)
  rows.append(row_last)
  table_data = []

  for row in rows:
    j=0
    t_row = {}
    for td in row.find_all("td")[1:]: # bỏ qua '#STT'
      if j == len(cols): # bỏ thẻ 'td' thừa
        break
      t_row[cols[j]] = td.text.replace('\n', '').strip()
      j+=1
    table_data.append(t_row)
  df = pd.DataFrame(table_data) # convert table_data to dataframe
  return df

In [11]:
which = ['main_table_countries_today', # today
         'main_table_countries_yesterday', # yesterday
         'main_table_countries_yesterday2'] # 2 days ago

for i in range(len(which)):
  d = (datetime.today() - timedelta(days=i)).strftime('%d-%m-%Y')  
  df = get_table(which[i])
  df.to_csv(f'./Worldometer-data/table_{d}_raw.csv')     

---

# Cào data-continent cho mỗi country để phục vụ cho quy trình preprocess

In [12]:
if not os.path.exists('./continents.csv'):
  table = soup.find('table', {'id': 'main_table_countries_today'})
  row_0 = table.tbody.find_all('tr', {'style': ''})[0]
  row_last = table.find('tbody', {'class': 'total_row_body body_world'})
  rows = table.tbody.find_all('tr', {'style': re.compile(r'(^$|^background)')})
  rows.insert(0, row_0)
  rows.append(row_last)
  continent = {}

  for row in rows:
    row_info = row.find_all("td")[1:]
    country = row_info[0].text.replace('\n', '').strip()

    for td in row_info: 
      try:
        if td['data-continent'] != '': 
          continent[country] = td['data-continent']
          break
      except:
        continue
  continent

  continent_df = pd.DataFrame.from_dict(continent, orient='index', columns=['continent'])
  continent_df.to_csv('./continents.csv')